In [1]:
# ! scp -r arc:/data/public/newsgroup20  ./data/

alt.atheism.txt                               100% 4950KB  10.9MB/s   00:00    
comp.graphics.txt                             100% 4078KB  11.3MB/s   00:00    
comp.os.ms-windows.misc.txt                   100% 4743KB  12.1MB/s   00:00    
comp.sys.ibm.pc.hardware.txt                  100% 2415KB  10.3MB/s   00:00    
comp.sys.mac.hardware.txt                     100% 2155KB  10.2MB/s   00:00    
comp.windows.x.txt                            100% 3745KB  14.4MB/s   00:00    
list.csv                                      100%   15KB 959.7KB/s   00:00    
misc.forsale.txt                              100% 1883KB  15.5MB/s   00:00    
rec.autos.txt                                 100% 2603KB  15.7MB/s   00:00    
rec.motorcycles.txt                           100% 2442KB  15.9MB/s   00:00    
rec.sport.baseball.txt                        100% 2766KB  17.3MB/s   00:00    
rec.sport.hockey.txt                          100% 3520KB  17.2MB/s   00:00    
sci.crypt.txt                           

In [2]:
# %load nbheader.py
%reload_ext autoreload
%autoreload 2

from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import col as S
from pyspark.sql import DataFrame, Row, Window
import os
import sys
import json
import datetime
import re
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# spark = SparkSession.builder.master("local[4]").getOrCreate()
# spark.getActiveSession()
# spark.stop()



In [3]:
spark = SparkSession.builder \
    .master("local[4]") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()
spark.getActiveSession()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/24 19:59:22 WARN Utils: Your hostname, RCBM8368-DIII.local, resolves to a loopback address: 127.0.0.1; using 10.250.94.128 instead (on interface en0)
25/09/24 19:59:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/24 19:59:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/24 19:59:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52461)
Traceback (most recent call last):
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Users/pmolnar/.b

In [5]:
NEWS_DATA = 'data/newsgroup20/'

In [6]:
df = spark.read.text(NEWS_DATA)
df.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
sc = spark.sparkContext

rdd = sc.binaryFiles(NEWS_DATA)
rdd.take(1)

# Convert RDD to (file_path, string_content)
rdd_strings = rdd.map(lambda x: (x[0], x[1].decode("utf-8", errors="replace")))

df = spark.createDataFrame(rdd_strings, schema=["file_path", "content"])

strlen = F.udf(lambda s: len(s) if s else 0, T.StringType())

df.printSchema()
df.select('file_path', strlen('content').alias('length')) \
    .show(3, truncate=False)

root
 |-- file_path: string (nullable = true)
 |-- content: string (nullable = true)



+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|file_path                                                                                                                                                               |length |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|file:/Users/pmolnar/Classes-Workshops/MSA8395_Special_Topics_in_Analytics/SpecialTopicsInAnalytics/apache_spark/spark_project/data/newsgroup20/sci.crypt.txt            |4174828|
|file:/Users/pmolnar/Classes-Workshops/MSA8395_Special_Topics_in_Analytics/SpecialTopicsInAnalytics/apache_spark/spark_project/data/newsgroup20/comp.sys.mac.hardware.txt|2206754|
|file:/Users/pmolnar/Classes-Workshops/MSA8395_Special_Topics_in_Analytics/SpecialTopicsInAnalytics/apach

In [30]:
! head -30 /Users/pmolnar/Classes-Workshops/MSA8395_Special_Topics_in_Analytics/SpecialTopicsInAnalytics/apache_spark/spark_project/data/newsgroup20/sci.crypt.txt 

Newsgroup: sci.crypt
document_id: 14147
From: Marc VanHeyningen <mvanheyn@cs.indiana.edu>
Subject: RIPEM Frequently Asked Questions

Archive-name: ripem/faq
Last-update: Sun, 7 Mar 93 21:00:00 -0500

ABOUT THIS POSTING
------------------
This is a (still rather rough) listing of likely questions and
information about RIPEM, a program for public key mail encryption.  It
(this FAQ, not RIPEM) was written and will be maintained by Marc
VanHeyningen, <mvanheyn@whale.cs.indiana.edu>.  It will be posted to a
variety of newsgroups on a monthly basis; follow-up discussion specific
to RIPEM is redirected to the group alt.security.ripem.

This month, I have reformatted this posting in an attempt to comply
with the standards for HyperText FAQ formatting to allow easy
manipulation of this document over the World Wide Web.  Let me know
what you think.

DISCLAIMER
----------
Nothing in this FAQ should be considered legal advice, or anything
other than one person's opinion.  If you want real legal ad

In [38]:

# from pyspark.sql.functions import explode, udf
# from pyspark.sql.types import ArrayType, StructType, StructField, StringType

# Define UDF schema for output columns
schema = T.ArrayType(
    T.StructType([
        T.StructField("news_group", T.StringType()),
        T.StructField("document_id", T.StringType()),
        T.StructField("from", T.StringType()),
        T.StructField("subject", T.StringType()),
        T.StructField("archive_name", T.StringType()),
        T.StructField("message", T.StringType()),
    ])
)

# Python function for header splitting and extraction
# def split_messages(content):
#     # Regex to match the header and capture fields
#     pattern = re.compile(
#         r"Newsgroup: ([^\n]+)\ndocument_id: ([^\n]+)\nFrom: ([^\n]+)\nSubject: (.+?)\n((?:.|\n)*?)(?=Newsgroup: |\Z)",
#         re.MULTILINE
#     )
#     results = []
#     for match in pattern.finditer(content):
#         news_group = match.group(1)
#         document_id = match.group(2)
#         sender = match.group(3)
#         subject = match.group(4)
#         message = match.group(5).strip()
#         results.append((news_group, document_id, sender, subject, message))
#     return results


def split_messages(content):
    # Updated regex with optional Archive-name
    pattern = re.compile(
        r"Newsgroup:\s*([^\n]+)\n"
        r"document_id:\s*([^\n]+)\n"
        r"From:\s*([^\n]+)\n"
        r"Subject:\s*(.+?)\n"   # Capture until end of Subject line
        r"\s*\n"               # Blank line (mandatory)
        r"(?:Archive-name:\s*([^\n]+)\n)?"  # Optional Archive-name
        r"((?:(?!Newsgroup: ).|\n)*?)"      # Non-greedy capture until next header or EOF
        r"(?=Newsgroup: |\Z)",              # Lookahead for next message or end
        re.MULTILINE
    )
    results = []
    for match in pattern.finditer(content):
        news_group = match.group(1)
        document_id = match.group(2)
        sender = match.group(3)
        subject = match.group(4)
        archive_name = match.group(5)
        message = match.group(6).strip()
        results.append((news_group, document_id, sender, subject, archive_name, message))
    return results

# Register UDF
split_messages_udf = F.udf(split_messages, schema)

fpath_basename = F.udf(lambda pth: os.path.basename(pth), T.StringType())

# Usage in DataFrame pipeline ("content" is the column with raw message text)
df_split = df \
    .withColumn("file_name", fpath_basename("file_path")) \
    .withColumn("messages_array", split_messages_udf(df["content"]))
df_exploded = df_split.select("file_name", F.explode("messages_array").alias("msg"))


final_df = df_exploded.select(
    "file_name",
    "msg.news_group",
    "msg.document_id",
    "msg.from",
    "msg.subject",
    "msg.archive_name",
    "msg.message"
)
# final_df.show(truncate=False)

display(final_df.limit(20).toPandas())

,file_name,news_group,document_id,from,subject,archive_name,message
0,sci.crypt.txt,sci.crypt,14147,Marc VanHeyningen <mvanheyn@cs.indiana.edu>,RIPEM Frequently Asked Questions,ripem/faq,"Last-update: Sun, 7 Mar 93 21:00:00 -0500\n\nA..."
1,sci.crypt.txt,sci.crypt,14831,mvanheyn@cs.indiana.edu (Marc VanHeyningen),RIPEM Frequently Asked Questions,ripem/faq,Last-update: 31 Mar 93 22:00:00 -0500\n\nABOUT...
2,sci.crypt.txt,sci.crypt,14832,mvanheyn@cs.indiana.edu (Marc VanHeyningen),RIPEM Frequently Noted Vulnerabilities,ripem/attacks,Last-update: 31 Mar 93 21:00:00 -0500\n\nSOME ...
3,sci.crypt.txt,sci.crypt,14982,Jim-Miller@suite.com,Certifying Authority question answered.,None,">>If you have access to FTP, try FTPing to rsa..."
4,sci.crypt.txt,sci.crypt,14983,C445585@mizzou1.missouri.edu (John Kelsey),"""Rubber-hose cryptanalysis""",None,Some sick part of me really liked that phrase....
5,sci.crypt.txt,sci.crypt,14984,grady@netcom.com (1016/2EF221),Declassifying media,None,There are many Urban Legends (maybe this ought...
6,sci.crypt.txt,sci.crypt,14985,gtoal@gtoal.com (Graham Toal),Re: Hard drive security for FBI targets,None,From: res@colnet.cmhnet.org (Rob Stampfli)\n\n...
7,sci.crypt.txt,sci.crypt,14986,warlord@MIT.EDU (Derek Atkins),Re: disk safety measure?,None,-----BEGIN PGP SIGNED MESSAGE-----\n\nIn artic...
8,sci.crypt.txt,sci.crypt,14987,neuhaus@bloch.informatik.uni-kl.de (Stephan Ne...,Re: PGP 2.2: general comments,None,neuhaus@vier.informatik.uni-kl.de (Stephan Neu...
9,sci.crypt.txt,sci.crypt,14988,WHMurray@DOCKMASTER.NCSC.MIL,Licensing.....,None,>This thread brings up the more general questi...
